In [16]:
from okx.api import Account
from okx.api import Trade
from okx_api import Market as Market_api
from pprint import pprint
import json
import os
import time
import math
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
KEY = os.getenv('key')
SECRET = os.getenv('secret')
PASSPHRASE = os.getenv('passphrase')

In [18]:
market = Market_api(key = '', secret = '', passphrase = '', flag = '0')
account = Account(
    # 交易账户需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)
# 实例化trade
trade = Trade(
    # 交易模块需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)

## 獲取特定貨幣的歷史價格

In [ ]:
result = market.get_history_index_candles(instId = "INJ-USDT", bar = "1m", limit = "10")
pprint(result['data'])
    
val = (float(result['data'][0][1]) - float(result['data'][9][1]))/float(result['data'][9][1])*100 #最新資料減第十個資料漲跌百分比
result['data'][1]# ts, 開盤, 最高, 最低, 收盤, k線狀態(0未完結, 1完結)
# 如果是跌幅
# float(result['data'][9][1])*(1 - abs(val)/100)
# 如果是漲幅
# float(result['data'][2][1])*(1 + abs(val)/100)

## 最大可買賣數量

In [5]:
max_size_result = account.get_max_size(instId='INJ-USDT', tdMode='cash')
max_size_result

{'code': '0',
 'data': [{'ccy': '',
   'instId': 'INJ-USDT',
   'maxBuy': '2.3510114105581062',
   'maxSell': '0'}],
 'msg': ''}

## 獲取帳戶資金

In [6]:
balance_result = account.get_balance(ccy='USDT')
balance_result['data'][0]['details'][0]['availBal']

'93.0553826413004'

## 獲取全倉槓桿設定

In [7]:
# 永续合约 BTC-USDT-SWAP 全仓
leverage_info_result = account.get_leverage_info(instId='INJ-USDT-SWAP', mgnMode='cross')
leverage_info_result

{'code': '0',
 'data': [{'instId': 'INJ-USDT-SWAP',
   'lever': '1',
   'mgnMode': 'cross',
   'posSide': 'long'},
  {'instId': 'INJ-USDT-SWAP',
   'lever': '1',
   'mgnMode': 'cross',
   'posSide': 'short'}],
 'msg': ''}

## 設定槓桿

In [8]:
set_leverage_result = account.set_leverage(
    instId='INJ-USDT-SWAP', 
    mgnMode='isolated', 
    lever='2.5', 
    posSide='long'
)
set_leverage_result

{'code': '0',
 'data': [{'instId': 'INJ-USDT-SWAP',
   'lever': '2.5',
   'mgnMode': 'isolated',
   'posSide': 'long'}],
 'msg': ''}

## 開倉

In [ ]:
# 以2U的价格，逐仓限价开多1张
set_order_swap_limit_open_result = trade.set_order(
    instId = 'INJ-USDT-SWAP',
    ordType = 'limit',
    tdMode = 'isolated',
    posSide = 'long',
    side = 'buy',
    sz = '3', # 已成交 | 委託總量
    px = '1', # 成交均價 | 委託價
)
set_order_swap_limit_open_result

## 獲取訂單

In [ ]:
order_result = trade.get_order(
    instId = 'BTC-USDT-SWAP',
    ordId = set_order_swap_limit_open_result['data'][0]['ordId']
)
order_result['data']#[0]['state']

## 平倉

In [ ]:
# 以2U的价格，逐仓限价平多1张个MANA-USDT-SWAP
set_order_swap_limit_close_result = trade.set_order(
    instId='MANA-USDT-SWAP',
    ordType='limit',
    tdMode='isolated',
    posSide='long',
    side='sell',
    sz='1',
    px='1',
)
set_order_swap_limit_close_result

## 取消訂單

In [19]:
# 按照instId與ordId取消訂單
set_cancel_order_result = trade.set_cancel_order(
    instId = 'INJ-USDT-SWAP', ordId = orders_pending_result['data'][0]['ordId']
)
set_cancel_order_result

## 獲取交易位承擔的訂單列表

In [25]:
orders_pending_result = trade.get_orders_pending()
# 幣種
orders_pending_result['data'][0]['instId']
# 槓桿
orders_pending_result['data'][0]['lever']
# ordId
orders_pending_result['data'][0]['ordId']

'667107219108282369'

In [42]:
orders_pending_result = trade.get_orders_pending()
orders_pending_result['data']#[0]

[]

## 市價全平

In [13]:
# 逐倉 多倉 市價全平倉
set_close_position_result = trade.set_close_position(
    instId = 'INJ-USDT-SWAP',
    posSide = 'long',
    mgnMode = 'isolated'
)
set_close_position_result

{'code': '51023', 'data': [], 'msg': "Position doesn't exist"}

## 查看持倉

In [43]:
positions_result = account.get_positions(instType='SWAP')
# 幣種
positions_result['data'][0]['instId']
# 槓桿
positions_result['data'][0]['lever']

positions_result['data']

[{'adl': '1',
  'availPos': '15',
  'avgPx': '38.9246666666666667',
  'baseBal': '',
  'baseBorrowed': '',
  'baseInterest': '',
  'bePx': '38.9636108054027',
  'bizRefId': '',
  'bizRefType': '',
  'cTime': '1705329989253',
  'ccy': 'USDT',
  'closeOrderAlgo': [],
  'deltaBS': '',
  'deltaPA': '',
  'fee': '-0.0291935',
  'fundingFee': '0',
  'gammaBS': '',
  'gammaPA': '',
  'idxPx': '38.507',
  'imr': '',
  'instId': 'INJ-USDT-SWAP',
  'instType': 'SWAP',
  'interest': '',
  'last': '38.529',
  'lever': '5',
  'liab': '',
  'liabCcy': '',
  'liqPenalty': '0',
  'liqPx': '27.44040919771736',
  'margin': '17.516',
  'markPx': '38.525',
  'mgnMode': 'isolated',
  'mgnRatio': '41.81947405828002',
  'mmr': '0.37561875',
  'notionalUsd': '57.777098249999995',
  'optVal': '',
  'pendingCloseOrdLiabVal': '',
  'pnl': '0',
  'pos': '15',
  'posCcy': '',
  'posId': '666778499323543553',
  'posSide': 'long',
  'quoteBal': '',
  'quoteBorrowed': '',
  'quoteInterest': '',
  'realizedPnl': '-0.0

In [46]:
positions_result = account.get_positions(instType='SWAP')
# 幣種
positions_result['data'][0]['instId']
# 槓桿
positions_result['data'][0]['lever']

positions_result['data'][0]['margin']

'35.26142'

In [49]:
balance_result = account.get_balance(ccy = 'USDT')
bal = balance_result['data'][0]['details'][0]['availBal']
print(f"USDT balance: {bal}")

USDT balance: 55.4843162713004


In [50]:
float(positions_result['data'][0]['margin'])/float(bal)

0.6355204924502095

In [61]:
bal = 100

first = bal*0.25
new_bal = bal - first
second = new_bal*0.3333
new_bal_second = bal - second - first
new_bal_second

50.0025

In [66]:
round(50/new_bal, 2)

0.67

In [72]:
round(0/new_bal_second, 2)# == 1

0.0